<a href="https://colab.research.google.com/github/kumar045/Assignment-For-Filed/blob/main/Elastic_Search_Database.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
from transformers import pipeline
import operator
from elasticsearch import Elasticsearch
from datetime import datetime
from PyPDF2 import PdfReader
import hashlib

# Step 0: Extract Text from PDF
def extract_text_from_pdf(pdf_path):
    pdf_reader = PdfReader(open(pdf_path, "rb"))
    text = ""
    for page_num in range(len(pdf_reader.pages)):
        page = pdf_reader.pages[page_num]
        text += page.extract_text()
    return text

# Step 1: Zero-Shot Classification to Determine Topic
def classify_document(document_text):
    classifier = pipeline("zero-shot-classification")
    candidate_labels = ["education", "politics", "business", "crime"]
    results = classifier(document_text, candidate_labels)
    max_index, _ = max(enumerate(results['scores']), key=operator.itemgetter(1))
    top_label = results['labels'][max_index]
    return top_label

# Step 2 and 3: Generate Document ID and Index Name
def generate_doc_id_and_index(top_label, document_text, file_name, timestamp):
    unique_string = f"{document_text}{file_name}{timestamp}"
    sha256_hash = hashlib.sha256(unique_string.encode()).hexdigest()
    doc_id = f"{top_label[0].upper()}{sha256_hash[:8]}"  # Using first 8 characters of hash for brevity
    index_name = f"{top_label}"
    return doc_id, index_name

# Step 4: Index Document in Elasticsearch
def index_document_in_elasticsearch(doc_id, index_name, document_text, file_name, timestamp):

    document = {
        "text": document_text,
        "file_name": file_name,
        "timestamp": timestamp
    }
    response = client.index(
        index=index_name,
        id=doc_id,
        document=document
    )
    return response

if __name__ == "__main__":
    # Path to your PDF file
    pdf_path = "/content/DWSample2-PDF.pdf"

    # Extract text from PDF
    document_text = extract_text_from_pdf(pdf_path)

    # Classify the document
    top_label = classify_document(document_text)

    # Additional metadata
    file_name = pdf_path.split("/")[-1]  # Extracting file name from path
    timestamp = datetime.now().strftime('%Y-%m-%dT%H:%M:%S')

    # Generate a unique document ID and index name
    doc_id, index_name = generate_doc_id_and_index(top_label, document_text, file_name, timestamp)

    # Index the document in Elasticsearch
    response = index_document_in_elasticsearch(doc_id, index_name, document_text, file_name, timestamp)

    print(f"Document indexed with ID: {doc_id} in index: {index_name}")
    print(f"Elasticsearch response: {response}")


No model was supplied, defaulted to facebook/bart-large-mnli and revision c626438 (https://huggingface.co/facebook/bart-large-mnli).
Using a pipeline without specifying a model name and revision in production is not recommended.


Document indexed with ID: B056422fd in index: business
Elasticsearch response: {'_index': 'business', '_id': 'B056422fd', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 2, 'failed': 0}, '_seq_no': 0, '_primary_term': 1}


In [12]:
from elasticsearch import Elasticsearch

# Step 1: Search Elasticsearch to get document ID and index
def search_document(query, index_name):
    response = client.search(index=index_name, body={
        "query": {
            "match": query
        }
    })
    if response['hits']['total']['value'] > 0:
        return response['hits']['hits'][0]['_id'], response['hits']['hits'][0]['_index']
    else:
        return None, None

# Step 2: Retrieve the document using the document ID and index name
def retrieve_document(doc_id, index_name):
    response = client.get(index=index_name, id=doc_id)
    return response['_source']

if __name__ == "__main__":
    # Your query and index
    query = {"text": "Featured on the Food Network"}
    index_name = "business"

    # Search for the document
    doc_id, found_index_name = search_document(query, index_name)

    if doc_id and found_index_name:
        # Retrieve the document
        document = retrieve_document(doc_id, found_index_name)

        # Extract file name
        file_name = document.get("file_name", "")

        if file_name:
            print(f"The text belongs to the document named {file_name}.")
        else:
            print("File name not found in the document.")
    else:
        print("No document found based on the search criteria.")



The text belongs to the document named DWSample2-PDF.pdf.


<ipython-input-12-ec7eb38a61f6>:5: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  response = client.search(index=index_name, body={


In [14]:
from transformers import pipeline
import operator
from elasticsearch import Elasticsearch

# Function to classify text and determine index name
def classify_text(text):
    classifier = pipeline("zero-shot-classification")
    candidate_labels = ["education", "politics", "business", "crime"]
    results = classifier(text, candidate_labels)
    max_index, _ = max(enumerate(results['scores']), key=operator.itemgetter(1))
    top_label = results['labels'][max_index]
    return top_label

# Search Elasticsearch to get document ID and index
def search_document(query, index_name):
    response = client.search(index=index_name, body={
        "query": {
            "match": query
        }
    })
    if response['hits']['total']['value'] > 0:
        return response['hits']['hits'][0]['_id'], response['hits']['hits'][0]['_index']
    else:
        return None, None

# Retrieve the document using the document ID and index name
def retrieve_document(doc_id, index_name):
    response = client.get(index=index_name, id=doc_id)
    return response['_source']

if __name__ == "__main__":
    # Your query text
    query_text = "Featured on the Food Network"

    # Classify the query text to determine the index name
    top_label = classify_text(query_text)
    index_name = f"{top_label}"

    # Search for the document
    doc_id, found_index_name = search_document({"text": query_text}, index_name)

    if doc_id and found_index_name:
        # Retrieve the document
        document = retrieve_document(doc_id, found_index_name)

        # Extract file name
        file_name = document.get("file_name", "")

        if file_name:
            print(f"The text belongs to the document named {file_name}.")
        else:
            print("File name not found in the document.")
    else:
        print("No document found based on the search criteria.")


No model was supplied, defaulted to facebook/bart-large-mnli and revision c626438 (https://huggingface.co/facebook/bart-large-mnli).
Using a pipeline without specifying a model name and revision in production is not recommended.


The text belongs to the document named DWSample2-PDF.pdf.


<ipython-input-14-d482e668cab0>:16: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  response = client.search(index=index_name, body={


In [15]:
# Function to classify text and determine index name
def classify_text(text):
    classifier = pipeline("zero-shot-classification")
    candidate_labels = ["education", "politics", "business", "crime"]
    results = classifier(text, candidate_labels)
    max_index, _ = max(enumerate(results['scores']), key=operator.itemgetter(1))
    top_label = results['labels'][max_index]
    return top_label

# Search Elasticsearch to find documents related to query text
def search_related_documents(query_text):
    # Classify the query text to determine the index name
    top_label = classify_text(query_text)
    index_name = f"{top_label}"

    response = client.search(index=index_name, body={
        "query": {
            "match": {
                "text": query_text
            }
        }
    })

    # If there are matching documents, return their IDs and file names
    if response['hits']['total']['value'] > 0:
        related_documents = []
        for hit in response['hits']['hits']:
            doc_id = hit['_id']
            file_name = hit['_source'].get("file_name", "")
            related_documents.append((doc_id, file_name))
        return related_documents
    else:
        return []

if __name__ == "__main__":
    # Your query text
    query_text = "Featured on the Food Network"

    # Search for related documents
    related_documents = search_related_documents(query_text)

    if related_documents:
        print(f"Found {len(related_documents)} documents related to the query:")
        for doc_id, file_name in related_documents:
            print(f"Document ID: {doc_id}, File Name: {file_name}")
    else:
        print("No documents found related to the query.")


No model was supplied, defaulted to facebook/bart-large-mnli and revision c626438 (https://huggingface.co/facebook/bart-large-mnli).
Using a pipeline without specifying a model name and revision in production is not recommended.


Found 1 documents related to the query:
Document ID: B056422fd, File Name: DWSample2-PDF.pdf


<ipython-input-15-db035adfb7c8>:16: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  response = client.search(index=index_name, body={


In [16]:
from transformers import pipeline
import operator

classifier = pipeline("zero-shot-classification")

results = classifier(
  '''In her complaint, the woman alleged that the IAS officer raped her on the pretext of marriage in March, 2020. Back then, he was not an IAS officer, but promised to marry her once he is selected for the post, the complaint says.

"I bore the expenses for Yuvraj's IAS preparation and took care of him when he went into depression on getting selected as an IAS officer. He said his family would never disapprove of our marriage if he cleared the Union Public Service Commission (UPSC) exam," the woman said in her complaint.''',
  candidate_labels=["education", "politics", "business","crime"]
)

print(results)
# Get the index of the max score
max_index, max_value = max(enumerate(results['scores']), key=operator.itemgetter(1))

# Extract the label using the index
top_label = results['labels'][max_index]

print(top_label)

No model was supplied, defaulted to facebook/bart-large-mnli and revision c626438 (https://huggingface.co/facebook/bart-large-mnli).
Using a pipeline without specifying a model name and revision in production is not recommended.


{'sequence': 'In her complaint, the woman alleged that the IAS officer raped her on the pretext of marriage in March, 2020. Back then, he was not an IAS officer, but promised to marry her once he is selected for the post, the complaint says.\n\n"I bore the expenses for Yuvraj\'s IAS preparation and took care of him when he went into depression on getting selected as an IAS officer. He said his family would never disapprove of our marriage if he cleared the Union Public Service Commission (UPSC) exam," the woman said in her complaint.', 'labels': ['crime', 'business', 'politics', 'education'], 'scores': [0.7683209180831909, 0.13174334168434143, 0.06714367121458054, 0.03279207646846771]}
crime


In [1]:
# Installing Elasticsearch Python client
!pip install elasticsearch PyPDF2
!pip install datasets evaluate transformers[sentencepiece]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.5/395.5 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.1/143.1 kB 15.6 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.0.4
    Uninstalling urllib3-2.0.4:
      Successfully uninstalled urllib3-2.0.4
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [9]:
from elasticsearch import Elasticsearch
from elasticsearch import Elasticsearch

# Password for the 'elastic' user generated by Elasticsearch
ELASTIC_PASSWORD = "jHkppEE4Go9QnA2piAyjC2fZ"

# Found in the 'Manage Deployment' page
CLOUD_ID = "7a974c9b987b419c8d019d2f0185e544:dXMtY2VudHJhbDEuZ2NwLmNsb3VkLmVzLmlvOjQ0MyRjYWJhMmQwNDZhMDM0OTBlYTIyMjE0ODBmZjIxYjgxNCRiODlhZGEwNTM2YzU0NjgyODc0ZWFkZTVjNzhiNjIzYw=="

# Create the client instance
client = Elasticsearch(
    cloud_id=CLOUD_ID,
    basic_auth=("elastic", ELASTIC_PASSWORD)
)

# Successful response!
client.info()




ObjectApiResponse({'name': 'instance-0000000001', 'cluster_name': 'caba2d046a03490ea2221480ff21b814', 'cluster_uuid': 'biL8xjXxSfS64h8IQaOLag', 'version': {'number': '8.9.2', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': 'e8179018838f55b8820685f92e245abef3bddc0f', 'build_date': '2023-08-31T02:43:14.210479707Z', 'build_snapshot': False, 'lucene_version': '9.7.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [10]:
client.index(
    index="my_index",
    id="my_document_id",
    document={
        "foo": "foo",
        "bar": "bar",
    }
)

client.get(index="my_index", id="my_document_id")


client.search(index="my_index", query={
    "match": {
        "foo": "foo"
    }
})

ObjectApiResponse({'took': 0, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 0, 'relation': 'eq'}, 'max_score': None, 'hits': []}})